# VROOM testing without ORS
updated with vroom 1.8.0 vroom-express 0.7, supporting Shipments

docs:  https://github.com/VROOM-Project/vroom/blob/v1.8.0/docs/API.md

In [11]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint
from datetime import datetime
from datetime import date
import time

vroom_url = 'http://vroom:3000/optimization/'

vehicleProfile = "driving-car"

# DATI posizioni
pointsA = json.load(open('../data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('../data/mezzocorona.geojson'))['features'] #6 points

example1 = json.load(open('../data/example_1_mezzocorona.json')) #from vroom shipments example

for v in example1['vehicles']:
    v['profile']=vehicleProfile

center = [11.10260,46.22007]
zoom = 15

point_start = [11.10249,46.21579] #posizione iniziale veicoli
point_end = point_start.copy()

def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def sec2hour(sec):
    return str(datetime.fromtimestamp(sec).strftime("%H:%M"))

def hour2sec(hour = '00:00'):
    now = datetime.now()
    s = now.strftime("%d/%m/%Y ") + hour
    return int(time.mktime(datetime.strptime(s, "%d/%m/%Y %H:%M").timetuple()))

def locShift(loc, s):
    return [loc[0] + s, loc[1] + s]

#CUSTOM ICONS https://fontawesome.com/icons?d=gallery&p=2

def vehicle2Map(v):
    idv = str(v['id'])    
    loc = revcc(v['start'])
    cap = str(v['capacity'][0])

    if hasattr(v,'time_window'):
        times = j['time_window']
    else:
        times = []
    
    mText = "vehicle %s <br> capacity %s" % (idv, cap)
    
    if len(times) > 0:
        mText += "<br>times from: %s <br> to %s " % (sec2hour(times[0]), sec2hour(times[1]) )
    
    mColor = COLORS[idv]
    mark = folium.Marker(loc, tooltip=folium.Tooltip(mText,permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
    mark.add_to(Map)

def job2Map(j):
    idj = str(j['id'])    
    loc = revcc(j['location'])
    if hasattr(j,'time_window'):
        times = j['time_windows'][0]
    else:
        times = []
    
    mText = "job %s<br>" % (idj)
    #+"quantity %s <br> delivery time %s" % (delivery, toMin(service))
    
    if len(times) > 0:
            mText += "from: %s <br> to: %s " % ( sec2hour(times[0]), sec2hour(times[1]) )

    mark = folium.Marker(loc, tooltip=folium.Tooltip(mText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

def ship2Map(s):
    amount = str(s['amount'][0])
    
    shipText = "spedizione quantita %s <br>" % amount
    
    ids = str(s['pickup']['id'])
    loc = revcc(s['pickup']['location'])
    mText = "carico %s " % ids
#         if hasattr(s,'time_window'):
#             times = s['time_windows'][0]
#         else:
#             times = []        
    #     if len(times) > 0:
    #             mText += "from: %s <br> to: %s " % ( sec2hour(times[0]), sec2hour(times[1]) )
    mark = folium.Marker(loc, tooltip=folium.Tooltip(shipText+mText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='red'))
    mark.add_to(Map)
    ids = str(s['delivery']['id'])
    loc = revcc(s['delivery']['location'])
    mText = "scarico %s " % ids
    mark = folium.Marker(loc, tooltip=folium.Tooltip(shipText+mText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='red'))
    mark.add_to(Map)

    #+"quantity %s <br> delivery time %s" % (delivery, toMin(service))
    

Map = folium.Map(location = revcc(center), tiles='OpenStreetMap', zoom_start = zoom)



COLORS = {
    '1': 'blue',
    '2': 'red',
    '3': 'purple',
    '4': 'green'
}

HOURS = [
    [
        hour2sec('08:00'),
        hour2sec('08:05')
    ],
    [
        hour2sec('09:00'),
        hour2sec('09:05')
    ],
    [
        hour2sec('10:00'),
        hour2sec('10:05')
    ],
    [
        hour2sec('11:00'),
        hour2sec('11:05')
    ],
    [
        hour2sec('12:00'),
        hour2sec('12:05')
    ]
]
# VEICOLI

vehicles = []

### VEICOLO 1
vid = 1
capacity = 1

timeWindow = [       #orario lavoro del veicolo 1
        hour2sec('08:00'),
        hour2sec('13:00')
    ]

loc = locShift(point_start, 0.004)

vehicle = {
    'id': vid,
    'capacity': [capacity],    # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,       # posizione di partenza veicoli
    'end': loc,
    'time_window': timeWindow,
#TODO
#     'breaks': [              #se ci sono orari precisi di pause e sanitizzazione del veicolo
#         {
#           "id": 1,
#           #"description": "pausa sanitizzazione", #descrizione pausa
#           "service": 300,   #durata pausa 5 minuti
#           "time_windows": [ #an array of time_window objects describing valid slots for break start
#             [
#               hour2sec('13:05'),
#               hour2sec('13:55')
#             ],
#           ]
#         }
#       ]
}
vehicles.append(vehicle)

### VEICOLO 2
vid = 2
capacity = 1

timeWindow = [                  # orario lavoro del veicolo 2
        hour2sec('14:00'),
        hour2sec('19:00')
    ]

loc = locShift(point_start, 0.003)

vehicle = {
    'id': vid,
    'capacity': [capacity],     # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,        # posizione di partenza veicoli
    'end': loc,
    #'time_window': timeWindow
}
vehicles.append(vehicle)

#DEFINITION OF JOBS

ccA = [ p['geometry']['coordinates'] for p in pointsA ]
ccB = [ p['geometry']['coordinates'] for p in pointsB ]
job_locs = ccA + ccB

jobs = []
for jid, cc in enumerate(job_locs):
    
    #quantita da consegnare
    delivery = 1 #random.randrange(1, 3)
    
    #tempo di della singola consegna/ritiro(in secondi)
    service = random.randrange(60, 300) #da 2 a 5 minuti
    
    #orario casuali
    timeWindow = random.choice(HOURS)
    
    # shifting casuale del tempo per ogni job
    #s = random.randrange(-3000,3000)
    #timeWindow = [timeWindow[0]+s, timeWindow[1]+s]
    job = {
        'id': jid,
        'location': cc,
        'delivery': [delivery], # Quantita' da consegnare
        #'service': service,      # tempo di consegna o aggiunta del tempo DI PULIZIA mezzo
        'time_windows': [ timeWindow ]
    }
    jobs.append(job)

# request = {
#     'vehicles': vehicles,
#     'jobs': jobs
# }

request = example1

# RENDERIZZA PROBLEMA RICHIESTO IN MAP
for vehicle in request['vehicles']:
    vehicle2Map(vehicle)

for job in request['jobs']:
    job2Map(job)

for ship in request['shipments']:
    ship2Map(ship)

Map

In [12]:
request

{'vehicles': [{'id': 1,
   'start': [11.10274, 46.21997],
   'end': [11.10274, 46.21997],
   'capacity': [4],
   'skills': [1, 14],
   'time_window': [1600416000, 1600430400],
   'profile': 'driving-car'},
  {'id': 2,
   'start': [11.10374, 46.21897],
   'end': [11.10374, 46.21897],
   'capacity': [4],
   'skills': [2, 14],
   'time_window': [1600416000, 1600430400],
   'breaks': [{'id': 2,
     'service': 300,
     'time_windows': [[1600423200, 1600425000]]}],
   'profile': 'driving-car'}],
 'jobs': [{'id': 1,
   'service': 300,
   'delivery': [1],
   'location': [11.114227983093262, 46.21858452155989],
   'skills': [1],
   'time_windows': [[1600419600, 1600423200]]},
  {'id': 2,
   'service': 300,
   'pickup': [1],
   'location': [11.118206977844238, 46.214911991592125],
   'skills': [1]}],
 'shipments': [{'amount': [1],
   'pickup': {'id': 3, 'location': [11.122541427612305, 46.20986352455638]},
   'delivery': {'id': 3,
    'location': [11.114301681518555, 46.20900226913922]}}]}

In [13]:
request['geometry'] = True       #RITORNA GEOMETRIA DELLA SOLUZIONE

resp = requests.post(vroom_url, json = request)

solution = resp.json()

#solution['summary']
solution

{'code': 2, 'error': 'Missing skills.'}

In [14]:
if len(solution['routes']) > 0:
    for route in solution['routes']:

        rtext = 'distance: '+toKm(route['distance'])+' delivered quantity: '+str(route['delivery'][0])+', time: '+toMin(route['duration']+route['service'])
        rcolor = COLORS[str(route['vehicle'])]
        print('Vehicle'+str(route['vehicle']), rcolor, rtext)

        folium.PolyLine(
            locations = [revcc(coords) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
            color = rcolor,
            opacity = 0.8,
            weight = 5
        ).add_to(Map)

    print("\nTotal distance: "+toKm(solution['summary']['distance']), 'Not Delivered: '+str(len(solution['unassigned'])))

Map

KeyError: 'routes'

In [15]:
solution

{'code': 2, 'error': 'Missing skills.'}